In [2]:
import sys
sys.path = ['/nfs/gns/homes/willj/anaconda3/envs/GTEx/lib/python3.5/site-packages'] + sys.path
import gzip
import pandas as pd
import numpy as np

In [16]:
with gzip.open('../../textfiles/Homo_sapiens.GRCh38.88.gtf.gz') as f:
    x = f.read().decode('utf-8')

In [58]:
z = [y.split('\t') for y in x.split('\n')[5:-1]]

In [61]:
z = [y for y in z if y[2] == 'transcript']

In [68]:
example_transcript = ENSG00000223972

In [7]:
tissue = 'Lung'

if tissue == 'Artery - Tibial':
    tissue_filename = 'Artery_Tibial'
elif tissue == 'Heart - Left Ventricle':
    tissue_filename = 'Heart_Left_Ventricle'
elif tissue == 'Breast - Mammary Tissue':
    tissue_filename = 'Breast_Mammary_Tissue'
elif tissue == 'Brain - Cerebellum':
    tissue_filename = 'Brain_Cerebellum'
else:
    tissue_filename = tissue

tissue_expression_filepath = '/nfs/research2/stegle/stegle_secure/GTEx/download/49139/PhenoGenotypeFiles/RootStudyConsentSet_phs000424.GTEx.v6.p1.c1.GRU/ExpressionFiles/phe000006.v2.GTEx_RNAseq.expression-data-matrixfmt.c1/parse_data/44_tissues/GTEx_Data_20150112_RNAseq_RNASeQCv1.1.8_gene_rpkm_{}_normalised_without_inverse_gene_expression.txt'.format(tissue_filename)

In [8]:

with open(tissue_expression_filepath, 'r') as f:
    expression_table = np.array([x.split('\t') for x in f.read().splitlines()])

I think it is difficult to extract this information from the faw gtf files. I will try to do so by using the ensembl API.

In [62]:
from pyensembl import EnsemblRelease

# release 77 uses human reference genome GRCh38
data = EnsemblRelease(77)

transcript_IDs = [x.split('.')[0] for x in expression_table[:,0][1:]]
gene_functions = []
for (i,ID) in enumerate(transcript_IDs):
    try:
        gene_functions.append(data.gene_by_id(ID).biotype)
    except ValueError:
        gene_functions.append('NA')
        
protein_coding_IDs = np.array(transcript_IDs)[np.array(gene_functions) == 'protein_coding']
protein_coding_indexes = np.array([transcript_IDs.index(x) for x in protein_coding_IDs])

In [54]:
pickle.dump(protein_coding_IDs, open('../../textfiles/lung_protein_coding_genes.py','wb'))

In [58]:
protein_coding_indexes = np.array([lung_transcripts.index(x) for x in protein_coding_IDs])

In [59]:
pickle.dump(protein_coding_indexes, open('../../textfiles/lung_protein_coding_indexes.py','wb'))